In [1]:
import matplotlib.pyplot as plt
import numpy as np
import glob
import cv2
import os

In [2]:
from clearml import Task
Task.init(project_name='pushup_training', task_name='notebook_training')

ClearML Task: created new task id=0c1cc6eee69d4bb79e2d5c20d3f0b157
ClearML results page: https://app.community.clear.ml/projects/b63fc03619e4435ea33998df02b9029c/experiments/0c1cc6eee69d4bb79e2d5c20d3f0b157/output/log


# Load the data

In [2]:
landmarks = []
landmark_labels = []

images = []
image_labels = []

for csv_name in sorted(glob.glob('../landmarks/**/*.csv')):
    landmarks.append(np.loadtxt(csv_name, delimiter=','))
    landmark_labels.append(os.path.basename(os.path.dirname(csv_name)))

for image_name in sorted(glob.glob('../raw_data/**/*.jpg')):
    images.append(cv2.imread(image_name))
    image_labels.append(os.path.basename(os.path.dirname(image_name)))

landmarks = np.array(landmarks)
landmark_labels = np.array(landmark_labels)
images = np.array(images)
image_labels = np.array(image_labels)

In [3]:
# These are all 33 landmarks, x and y summing up to 66. Augmentation will get rid of the unnessecary landmarks
landmarks.shape

(33, 35, 3)

In [4]:
landmark_labels.shape

(33,)

# Select only important landmarks

In [9]:
# Take only the useful keypoints (14)
selected_keypoints = [2, 5, 11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28]
# Keep only keypoints in the list; resulting shape: (-1, 14, 3). 3d coords for each selected keypoint
selected_landmarks = np.take(landmarks, selected_keypoints, axis=1)
# Remove the 3rd dim coordinate and flatten the x and y coords into 1 vector
selected_landmarks = np.array(selected_landmarks)[:, :, :2]
selected_landmarks = selected_landmarks.reshape(selected_landmarks.shape[0], -1)
selected_landmarks.shape

(33, 28)

# Training

In [10]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from confusion_matrix_pretty_print import plot_confusion_matrix_from_data

In [11]:
X_train, X_test, y_train, y_test, images_train, images_test, image_labels_train, image_labels_test = train_test_split(selected_landmarks, landmark_labels, images, image_labels, test_size=0.3, random_state=42)

## Augmentation

In [12]:
import os
if '..' not in os.sys.path:
    os.sys.path.append('..')

from pushup_lockscreen.augmentation import LandMarkAugmentation

In [13]:
augmenter = LandMarkAugmentation()
# add 10x the amount of original data in augmented versions
augmented_images, augmented_landmarks, original_landmarks = augmenter.get_augmented_batch(images, X_train.reshape(-1, len(selected_keypoints), 2))

# keypoint_image = augmented_landmarks[0].draw_on_image(augmented_images[0], size=7)
# plt.imshow(keypoint_image)
# plt.show()
# keypoint_image = original_landmarks[0].draw_on_image(images[0], size=7)
# plt.imshow(keypoint_image)
# plt.show()

IndexError: list index out of range

In [ ]:
# Normalize the landmarks of course
scaler = StandardScaler()
# Only fit the scaler on training data! Otherwise knowledge of test data will leak into training set
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
model = SVC(kernel='rbf', C=1, gamma=0.001)
model.fit(X_train, y_train)
preds = model.predict(X_test)
print(preds)
plot_confusion_matrix_from_data(y_test, preds, columns=['DOWN', 'UP'])

In [ ]:
X_train.shape